In [1]:
# Global import

import datetime as dt

import pandas as pd
import numpy as np
import plotly.graph_objects as go

from dash_mdc_neptune import themes

In [2]:
# Global variables

ASSET_NAME = "PWRTE"
PORTFOLIO_NAME = "TEST_FR"

# PAGE 1

In [3]:
# Retrieve trades

query_params = {
    "asset": ASSET_NAME,
    "portfolio_name": PORTFOLIO_NAME,
}

trades_df = nptc.filter_forward_contracts(**query_params)

In [4]:
# Format trades

all_df = trades_df[npta.definitions.TRADE_DATE_COLUMNS]
latest_df = all_df[all_df.traded_at==all_df.traded_at.max()]

## SECTION 1

### CARD 1

In [5]:
all_styler = all_df.style
all_styler = all_styler.set_table_styles(
    [
        themes.pandas.noos_pd_row_hover,
        themes.pandas.noos_pd_headers,
    ]
)
all_styler = all_styler.hide_index()

In [6]:
latest_styler = latest_df.style
latest_styler = latest_styler.set_table_styles(
    [
        themes.pandas.noos_pd_row_hover,
        themes.pandas.noos_pd_headers,
    ]
)
latest_styler = latest_styler.hide_index()

### CARD 2

In [7]:
trades_volume_df = npta.get_volumes_and_costs_from_trades(all_df)
latest_volumes_df = npta.get_volumes_and_costs_from_trades(latest_df)

In [29]:
# Shared heatmap configs

GO_LAYOUT = go.Layout(
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='local delivery date',
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='MWh',
        ),
        side="left",
        overlaying='y2',
    ),
    yaxis2=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='EUR/MWh',
        ),
        side="right",
    ),
    template="noos_watermark+noos_base+hoover_xunified+min_modebar",
)

In [30]:
all1 = go.Scatter(
    x = trades_volume_df.index,
    y = trades_volume_df.total_volume * 2,
    line_color=themes.NOOS_HTML_COLORS["gray"],
    name = "Volumes",
    fill='tozeroy',
    yaxis="y",
    visible = True
)

all2 = go.Scatter(
    x = trades_volume_df.index,
    y = trades_volume_df.price_average,
    line_color=themes.NOOS_HTML_COLORS["accent"],
    name = "Prices",
    visible = True,
    yaxis="y2"
)

latest1 = go.Scatter(
    x = latest_volumes_df.index,
    y = latest_volumes_df.total_volume * 2,
    line_color=themes.NOOS_HTML_COLORS["gray"],
    name = "Volumes",
    fill='tozeroy',
    yaxis="y",
    visible = False
)

latest2 = go.Scatter(
    x = latest_volumes_df.index,
    y = latest_volumes_df.price_average,
    line_color=themes.NOOS_HTML_COLORS["accent"],
    name = "Prices",
    visible = False,
    yaxis="y2"
)

fig = go.FigureWidget(
    data=[all1, all2, latest1, latest2],
    layout=GO_LAYOUT,
)

# DASH

In [31]:
import jupyter_dash

from dash import dcc

import dash_mdc_neptune as mdc

## LAYOUT

In [38]:
# Display graph & table in tabs
graph = dcc.Graph(
    id="graph",
    figure=fig,
    style={"height": "100%", "width": "100%"},
)
table = mdc.Table(
    id='table',
    columns=[{"field": c, "headerName": c, "width": 200} for i in all_df.columns],
    rows=all_df.to_dict(orient='records'),
    pageSize=10,
)

tab = mdc.Tab(
    id="tab",
    children=[table, graph],
    options=[
        {"label": "Trade blotter"},
        {"label": "Trade profile"},
    ],
)

In [39]:
# Visualize based on toggles
toggle = mdc.Toggle(
    id="toggle",
    options=[
        {'label': 'All'},
        {'label': 'Last'},
    ],
    selected='all',
)

box = mdc.Box(children=[toggle, tab])

In [40]:
# Dashboard layout
section = mdc.Section(
    children=box,
    options=[{"title": "Custom portfolio"}],
)

page = mdc.Page(children=section)
navbar = mdc.NavBar(title="Wholesale Hedge Trades")

layout = mdc.Dashboard(children=[navbar, page])

## APP

In [41]:
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output


app = JupyterDash(__name__)
app.layout = layout


@app.callback(
    Output(component_id='table', component_property='rows'),
    Output(component_id='graph', component_property='figure'),
    Input(component_id='toggle', component_property='selected')
)
def update_tabs(value):
    if value == 'all':
        rows = all_df.to_dict('records')
        fig.data[2].visible=False
        fig.data[3].visible=False
        fig.data[0].visible=True
        fig.data[1].visible=True
    else:
        rows = latest_df.to_dict('records')
        fig.data[0].visible=False
        fig.data[1].visible=False
        fig.data[2].visible=True
        fig.data[3].visible=True
    return [rows, fig]

app.run_server(mode='external', host="0.0.0.0", port=8001)

Dash app running on http://0.0.0.0:8001/


/opt/conda/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

